<a href="https://colab.research.google.com/github/takatakamanbou/Vision/blob/main/Vision2021_ex02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vision2021-ex02

課題の期限や提出の方法などについては，Visionチーム内に書いてます．

## 準備

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#####  最小二乗法の正規方程式を解く関数
### solving the linear least squares problem (1)
#
# X.shape is assumed to be (N, D+1)
# y.shape is assumed to be (N,)
#
def solve(X, y):

    A = np.dot(X.T, X)
    b = np.dot(X.T, y)

    # x is the solution of the equation Ax = b
    x = np.linalg.solve(A, b)

    return x


### solving the linear least squares problem (2)
#
def solve2(X, y):

    A = np.dot(X.T, X)
    b = np.dot(X.T, y)

    # rv[0] is the solution minimizing ||Ax - b||^2
    rv = np.linalg.lstsq(A, b)

    return rv[0]

In [ ]:
##### 1, x, x^2, x^3, ..., X^D をならべたデータ行列（N x (D+1)）をつくる
#     授業の説明のものとは転置の関係になっていることに注意
#
def makeDataMatrix(x, D):

    N = x.shape[0]
    X = np.zeros((N, D+1))
    X[:, 0] = 1
    for i in range(1, D+1):
        X[:, i] = x**i

    return X

## 課題A

以下のプログラムの中の関数 gendat は，ある3次関数にノイズをのせたデータを生成するものである．どんな3次関数を使っているか，プログラムを読み解きなさい．また，どんなノイズをのせているか，関数名などを頼りに詳しく調べなさい．

提出するもの： 実行結果のグラフと，上記について調べた結果

In [ ]:
def gendat(n, seed = 0, sigma = 0.0):

    x = np.linspace(-1.5, 2.5, num = n)
    y = x*x*x - x*x - x + 1.0
    
    np.random.seed(seed)
    y += sigma * np.random.randn(n)

    X = np.vstack((x, y)).T

    return X

In [ ]:
### 乱数の種(seed)を指定．適当に変えれば結果も変わる
seed = 0

data_noisy = gendat(30, seed=seed, sigma=0.5)
data_true  = gendat(1000, seed=seed, sigma=0)

fig, ax = plt.subplots(facecolor="white", figsize=(8, 6))
ax.set_xlim(-2, 3)
ax.set_ylim(-5, 10)
ax.plot(data_true[:, 0], data_true[:, 1], color="blue", label="true function")
ax.scatter(data_noisy[:, 0], data_noisy[:, 1], color="red", label="noisy data")
ax.legend()
plt.show()

## 課題B

以下のプログラムは，課題Aのプログラムを用いて生成したデータに D 次多項式をあてはめるものである．次のことをやりなさい．

1. 実行すると表示される mean squared error (MSE) が何を表しているか，プログラムを読解しなさい．
1. D を 1 から 20 まで変えたとき MSE の値やグラフがどのように変化するか観察しなさい．
1. 横軸を D，縦軸を学習データおよびテストデータの MSE としてグラフを描きなさい（Colab上でやってもよいし，他の手段を使ってもよい）．D が小さいときの MSE は他に比べてとても大きくなるので，縦軸の範囲を狭めてグラフを描き，D がいくつのときにテストデータの MSE が最小となっているか読み取れるようなものにすること．

提出するもの：上記の最後のステップで描いたグラフと，それに対する説明・考察．

**注意**
この課題と次の課題では，テストデータを使って次元数 $D$ や正則化パラメータ $\alpha$ の最適な値を決めようとしてます．授業で説明したように，未知のはずのテストデータを使うのは本当はいんちきです．



In [ ]:
##### generating the data

ndata = 30
sigma = 0.5

# learning (training) data
datL = gendat(ndata, seed=0, sigma=sigma)

# test data
datT = gendat(ndata, seed=1, sigma=sigma)

In [ ]:
##### fitting a D-th degree polynomial by least squares method

D = 1
XL = makeDataMatrix(datL[:, 0], D)
yL = datL[:, 1]
print('# X.shape =', XL.shape)
print('# y.shape =', yL.shape)
w = solve(XL, yL)
print('# estimated coefficients = ', w)

In [ ]:
##### mean squared error for the learning data

yL_est = np.dot(XL, w)
msqeL = np.mean((yL - yL_est)**2)
print('# mean squared error (L) = {:.3f}'.format(msqeL))


##### mean squared error for the test data

XT = makeDataMatrix(datT[:, 0], D)
yT = datT[:, 1]
yT_est = np.dot(XT, w)
msqeT = np.mean((yT - yT_est)**2)
print('# mean squared error (T) = {:.3f}'.format(msqeT))

In [ ]:
##### making the true (noiseless) data & more...

datTrue = gendat(1000)
X = makeDataMatrix(datTrue[:, 0], D)
y = datTrue[:, 1]
y_est = np.dot(X, w)


##### plotting the results

fig, ax = plt.subplots(facecolor="white", figsize=(8, 6))
ax.set_xlim(-2, 3)
ax.set_ylim(-5, 10)
ax.plot(datTrue[:, 0], datTrue[:, 1], color="blue", label="true function")
ax.scatter(datL[:, 0], datL[:, 1], color="red", label="noisy data")
ax.plot(datTrue[:, 0], y_est, color="green", label="estimated curve")
ax.legend()
plt.show()

## 課題C

上記の問題を，$L_2$ 正則化を用いて解いてみよう．次のことをやりなさい．

1. 関数 solve をもとにして，正則化を考慮した関数 solve3 を作りなさい．solve3 は X, y に加えて alpha という名前の引数で正則化項の重み $\alpha$ の値を受け取るようにしよう．パラメータのうち定数の係数（バイアス項）には正則化を効かせないようにすること．
1. D を 20 として，$\alpha$ を様々に変えて実験を行い，横軸を $\alpha$，縦軸を学習データおよびテストデータに対する誤差としてグラフを描いてみよう．
    - $\alpha$ は $0$ 以上の任意の値をとるので，いくつか適当な値を選んで実行 → 結果を見て範囲を変える／狭める，というステップを何回か繰り返すのがよい．とりあえず今回は [0, 10] からスタートしたらよい．
    - 余裕があれば，1つの $\alpha$ に対してノイズ生成の乱数の種を何通りか変えて，それらの誤差の平均でグラフを描くとなおよい
    - さらに余裕があれば，学習データの数を増減して同じ実験をやると傾向が変わるかどうか調べるとよい．
    - もっと余裕があれば，真の値に対する誤差も描いてみるとよい．

提出するもの：関数 solve3 の定義，描いたグラフとそれに対する説明・考察．